In [6]:
from selenium import webdriver
from bs4 import BeautifulSoup
import csv

In [7]:
url = 'https://forecast.weather.gov/MapClick.php?lat=37.777120000000025&lon=-122.41963999999996#.X5kUp1gzbtQ'
driver = webdriver.Chrome()
driver.get(url)

In [8]:
soup = BeautifulSoup(driver.page_source,'html.parser')

In [9]:
results = soup.find_all('div',{"class":"tombstone-container"})

In [13]:
# finding the weather description forecast from the other table
c_result = soup.find_all('div',{"class":"row-forecast"})

In [14]:
#The period description is different in the two tables so we will clean the text for matching them
def remove(string): 
    return string.replace(" ", "") 

In [15]:
def extract_record(item):
    """extract and return result from a single result"""
    period = item.p.text.strip()
    short_desc = item.find('p',{'class':'short-desc'}).text.strip()
    temp = item.find('p',{'class':'temp'}).text.strip()
    desc=''
    for c_item in c_result:
        if period==remove(c_item.find('div',{"class":"col-sm-2 forecast-label"}).text.strip()):
            desc =c_item.find('div',{"class":"col-sm-10 forecast-text"}).text.strip()
    result = (period,short_desc,temp,desc)
    return result

In [16]:
records = []
for item in results:
    record = extract_record(item)
    records.append(record)
driver.close()

In [17]:
records

[('Tonight',
  'Partly Cloudy',
  'Low: 53 °F',
  'Partly cloudy, with a low around 53. West southwest wind around 6 mph.'),
 ('Friday',
  'Mostly Sunny',
  'High: 71 °F',
  'Mostly sunny, with a high near 71. West wind 3 to 8 mph.'),
 ('FridayNight',
  'Partly Cloudy',
  'Low: 52 °F',
  'Partly cloudy, with a low around 52. West wind 5 to 9 mph.'),
 ('Saturday',
  'Mostly Sunny',
  'High: 74 °F',
  'Mostly sunny, with a high near 74. Calm wind becoming west around 6 mph in the afternoon.'),
 ('SaturdayNight',
  'Mostly Clear',
  'Low: 53 °F',
  'Mostly clear, with a low around 53. West southwest wind 5 to 9 mph.'),
 ('Sunday', 'Sunny', 'High: 76 °F', 'Sunny, with a high near 76.'),
 ('SundayNight',
  'Mostly Clear',
  'Low: 54 °F',
  'Mostly clear, with a low around 54.'),
 ('Monday', 'Sunny', 'High: 76 °F', 'Sunny, with a high near 76.'),
 ('MondayNight',
  'Partly Cloudy',
  'Low: 54 °F',
  'Partly cloudy, with a low around 54.')]

In [18]:
#save data in csv file
with open('weather_forcast.csv','w', newline='',encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['period', 'short_desc', 'temp', 'desc'])
    writer.writerows(records)